In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import s3fs  # Para interactuar con S3
import os  # Para manejar nombres de archivo/ruta
import tempfile
# No se necesita matplotlib.pyplot en este script, se puede eliminar si se desea.

def preprocess_and_save_s3(s3, input_s3_path, output_s3_path, selected_video_ids, target_resolution=(120, 160)):
    """
    Lee datos de video desde un archivo HDF5 en S3, los preprocesa
    (ajuste de longitud, redimensionado) y guarda el resultado en
    otro archivo HDF5 en S3.

    Args:
        s3 (s3fs.S3FileSystem): Instancia del sistema de archivos S3.
        input_s3_path (str): Ruta S3 completa al archivo HDF5 de entrada.
        output_s3_path (str): Ruta S3 completa donde se guardará el archivo HDF5 preprocesado.
        selected_video_ids (list): Lista de IDs de video a procesar del archivo de entrada.
        target_resolution (tuple): Resolución (altura, anchura) a la que redimensionar los frames.
    """
    print(f"Ruta de entrada: {input_s3_path}")
    print(f"Ruta de salida: {output_s3_path}")

    processed_data = {}  # Diccionario para almacenar datos procesados temporalmente {vid: data}
    glosses_map = {}  # Diccionario para almacenar los glosses {vid: gloss}
    target_frames = None  # Se determinará a partir de los datos

    try:
        # Abre el archivo HDF5 de entrada directamente desde S3 usando s3fs
        print("Accediendo al archivo HDF5 de entrada desde S3...")
        with s3.open(input_s3_path, 'rb') as s3_input_file:
            with h5py.File(s3_input_file, 'r') as f_in:
                print("Archivo HDF5 de entrada abierto.")

                # Calcula la longitud mediana de los frames
                frame_counts = [f_in[vid]['frames'].shape[0] for vid in selected_video_ids]
                print(f"Se encontraron {len(frame_counts)} secuencias de video.")
                target_frames = int(np.median(frame_counts))
                print(f"Mediana de frames (longitud estándar): {target_frames}")

                # Procesa cada video seleccionado
                for i, vid in enumerate(selected_video_ids):
                    print(f"\nProcesando video {i+1}/{len(selected_video_ids)} (ID: {vid})")
                    video_data = f_in[vid]['frames'][:]  # Carga los datos del video
                    gloss = f_in[vid].attrs['gloss']
                    glosses_map[vid] = gloss
                    
                    current_frames = video_data.shape[0]
                    
                    # --- Ajuste de longitud (Padding o Recorte) ---
                    adjusted_video_data = None
                    if current_frames < target_frames:
                        # print("Aplicando padding (interpolación)...")
                        indices = np.linspace(0, current_frames - 1, num=target_frames)
                        # Vectorización para mayor eficiencia
                        idx_lower = np.floor(indices).astype(int)
                        idx_upper = np.ceil(indices).astype(int)
                        # Asegurarse de que upper no exceda el límite
                        idx_upper[idx_upper >= current_frames] = current_frames - 1
                        weight = indices - idx_lower
                        
                        # Interpola usando los pesos. Añade dimensiones para broadcasting
                        adjusted_video_data = (1 - weight[:, np.newaxis, np.newaxis, np.newaxis]) * video_data[idx_lower] + \
                                              weight[:, np.newaxis, np.newaxis, np.newaxis] * video_data[idx_upper]
                        adjusted_video_data = adjusted_video_data.astype(video_data.dtype)

                    elif current_frames > target_frames:
                        # print("Aplicando recorte (muestreo uniforme)...")
                        indices = np.linspace(0, current_frames - 1, num=target_frames, dtype=int)
                        adjusted_video_data = video_data[indices]
                    
                    else:
                        # print("La longitud ya es igual a la mediana.")
                        adjusted_video_data = video_data

                    # --- Redimensionado de frames ---
                    if adjusted_video_data.ndim < 3:
                         print(f"Advertencia: El video {vid} tiene dimensiones inesperadas {adjusted_video_data.shape}. Se omite.")
                         continue
                    
                    # tf.image.resize espera float, convierte si es necesario
                    if adjusted_video_data.dtype != np.float32:
                        adjusted_video_data_float = adjusted_video_data.astype(np.float32)
                    else:
                        adjusted_video_data_float = adjusted_video_data

                    resized_video_data = tf.image.resize(adjusted_video_data_float, target_resolution).numpy()

                    # --- Añadir dimensión de canal si es necesario ---
                    if resized_video_data.ndim == 3:
                        resized_video_data = resized_video_data[..., np.newaxis]
                    
                    processed_data[vid] = resized_video_data

    except Exception as e:
        print(f"Error durante la lectura o procesamiento del archivo de entrada: {e}")
        raise

    # --- Guardar los datos preprocesados en un nuevo archivo HDF5 ---
    if not processed_data:
        print("No se procesaron datos, no se guardará ningún archivo de salida.")
        return None

    print(f"\nPreparando para guardar {len(processed_data)} videos preprocesados en: {output_s3_path}")
    local_temp_path = None
    try:
        with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp_h5:
            local_temp_path = tmp_h5.name
        
        with h5py.File(local_temp_path, 'w') as f_out:
            print("Archivo HDF5 temporal abierto para escritura local.")
            for vid, data in processed_data.items():
                grp = f_out.create_group(vid)
                grp.create_dataset('frames', data=data, dtype=data.dtype, compression='gzip')
                grp.attrs['gloss'] = glosses_map[vid]
        
        print("Subiendo archivo a S3...")
        s3.put(local_temp_path, output_s3_path)
        print("Archivo subido a S3 exitosamente.")

    except Exception as e:
        print(f"Error durante el guardado local o la subida a S3: {e}")
        raise
    finally:
        if local_temp_path and os.path.exists(local_temp_path):
            print(f"Eliminando archivo temporal local: {local_temp_path}")
            os.remove(local_temp_path)

    print("¡Preprocesamiento del dataset completado!")
    return output_s3_path

def main():
    """
    Función principal que orquesta el preprocesamiento de múltiples datasets.
    """
    # --- Rutas S3 ---
    input_s3_folder = "s3://representatiohorarum/Datasets/Completos/"
    output_s3_folder = "s3://representatiohorarum/Datasets/Luego del trim/"

    # --- Archivos a procesar (entrada -> salida) ---
    datasets = {
        "ISL_glosscomun.h5": "ISL_trim.h5",
        "SLOVO_glosscomun.h5": "SLOVO_trim.h5",
        "WLSL_V03_glosscomun.h5": "WLSL_trim.h5"
    }

    # Inicializa el sistema de archivos S3
    # s3fs usará las credenciales de AWS configuradas en el entorno
    s3 = s3fs.S3FileSystem()

    # Asegurarse de que el directorio de salida exista en S3
    if not s3.exists(output_s3_folder):
        print(f"Creando directorio de salida en S3: {output_s3_folder}")
        s3.makedirs(output_s3_folder)

    # --- Bucle de procesamiento ---
    for input_filename, output_filename in datasets.items():
        input_s3_path = os.path.join(input_s3_folder, input_filename)
        output_s3_path = os.path.join(output_s3_folder, output_filename)
        
        print(f"\n{'='*60}")
        print(f"INICIANDO PROCESAMIENTO PARA: {input_filename}")
        print(f"{'='*60}")

        try:
            # Obtener todos los IDs de video del archivo de entrada
            print("Obteniendo IDs de video del archivo de entrada...")
            with s3.open(input_s3_path, 'rb') as temp_s3_file:
                with h5py.File(temp_s3_file, 'r') as temp_h5_file:
                    ids_to_process = list(temp_h5_file.keys())
            
            if not ids_to_process:
                print(f"Advertencia: No se encontraron videos en {input_s3_path}. Saltando al siguiente.")
                continue

            print(f"Se procesarán {len(ids_to_process)} videos.")
            
            # Llamar a la función principal de preprocesamiento
            final_output_path = preprocess_and_save_s3(
                s3=s3,
                input_s3_path=input_s3_path,
                output_s3_path=output_s3_path,
                selected_video_ids=ids_to_process,
                target_resolution=(120, 160)  # Puedes cambiar la resolución aquí
            )

            if final_output_path:
                print(f"\nPROCESO EXITOSO para {input_filename}.")
                print(f"Archivo preprocesado guardado en: {final_output_path}")

        except Exception as e:
            print(f"\n{'!'*20} ERROR {'!'*20}")
            print(f"Ocurrió un error fatal procesando el archivo {input_s3_path}: {e}")
            print("Saltando al siguiente dataset si hay más.")
            print(f"{'!'*47}")
            continue # Continúa con el siguiente archivo en caso de error

    print(f"\n{'='*60}")
    print("TODOS LOS PROCESOS HAN FINALIZADO.")
    print(f"{'='*60}")


# --- Punto de entrada del script ---
if __name__ == "__main__":
    main()

In [ ]:
import h5py
import s3fs
import os
import matplotlib.pyplot as plt
import numpy as np

def inspect_h5_files_and_show_frames_from_s3():
    """
    Reads H5 files from S3, displays frames of the first 4 video sequences per gloss in each file,
    and prints summary information for each sequence.
    """
    # --- S3 Configuration ---
    s3_folder = "s3://representatiohorarum/Datasets/Luego del trim/"
    files_to_inspect = [
        "ISL_trim.h5",
        "SLOVO_trim.h5",
        "WLSL_trim.h5"
    ]

    print(f"Inspecting files in: {s3_folder}\n")

    try:
        s3 = s3fs.S3FileSystem()
    except Exception as e:
        print(f"Error initializing S3FileSystem. Ensure AWS credentials are configured. Error: {e}")
        return

    header = (
        f"{'Archivo Origen':<25} | "
        f"{'ID de Secuencia':<15} | "
        f"{'Etiqueta (Gloss)':<20} | "
        f"{'Dimensiones (F, H, W, C)':<28} | "
        f"{'Tipo de Dato'}"
    )
    print(header)
    print("-" * len(header))

    for filename in files_to_inspect:
        full_s3_path = os.path.join(s3_folder, filename)
        gloss_count = {}  # Dictionary to track number of videos per gloss

        try:
            with s3.open(full_s3_path, 'rb') as s3_file:
                with h5py.File(s3_file, 'r') as hf:
                    sequence_ids = list(hf.keys())
                    if not sequence_ids:
                        print(f"\nWarning: File {filename} is empty or contains no sequences.\n")
                        continue

                    for seq_id in sequence_ids:
                        sequence_group = hf.get(seq_id)
                        if sequence_group is None:
                            print(f"Warning: Sequence ID '{seq_id}' not found in {filename}.")
                            continue

                        label = sequence_group.attrs.get('gloss', 'N/A')

                        # Initialize or update gloss count
                        if label not in gloss_count:
                            gloss_count[label] = 0
                        gloss_count[label] += 1

                        # Process only if we haven't exceeded 4 videos for this gloss
                        if gloss_count[label] <= 4:
                            if 'frames' in sequence_group:
                                frames_dataset = sequence_group['frames']
                                shape = frames_dataset.shape
                                dtype = frames_dataset.dtype

                                row = (
                                    f"{filename:<25} | "
                                    f"{seq_id:<15} | "
                                    f"{label:<20} | "
                                    f"{str(shape):<28} | "
                                    f"{str(dtype)}"
                                )
                                print(row)

                                # Display all frames in a single row
                                num_frames = shape[0]
                                fig, axes = plt.subplots(1, num_frames, figsize=(num_frames * 4, 4))
                                if num_frames == 1:
                                    axes = [axes]  # Ensure axes is iterable for a single frame
                                for i, frame in enumerate(frames_dataset):
                                    if frame.ndim == 3 and frame.shape[-1] == 1:
                                        axes[i].imshow(frame[:, :, 0], cmap='gray')
                                    elif frame.ndim == 3:
                                        axes[i].imshow(frame)
                                    elif frame.ndim == 2:
                                        axes[i].imshow(frame, cmap='gray')
                                    else:
                                        print(f"Warning: Frame {i} in sequence {seq_id} has unexpected dimensions: {frame.shape}")
                                    axes[i].set_title(f"Frame {i}")
                                    axes[i].axis('off')  # Turn off axis numbers and ticks
                                plt.suptitle(f"{filename} - {seq_id} - Gloss: {label}")
                                plt.tight_layout()
                                plt.show()

                            else:
                                print(f"Warning: 'frames' dataset not found in sequence '{seq_id}' of {filename}.")

        except FileNotFoundError:
            print(f"\nERROR: File not found in S3: {full_s3_path}\n")
        except Exception as e:
            print(f"\nERROR: An error occurred processing {filename}: {e}\n")

    print("\nFinished inspecting all files.")

if __name__ == "__main__":
    inspect_h5_files_and_show_frames_from_s3()

In [ ]:
!pip install ultralytics

In [ ]:
# --- Imports necesarios (asegúrate de tenerlos en tu script) ---
import cv2
import numpy as np
import h5py
import s3fs
import os
import io # <--- IMPORTANTE: Necesario para la solución de guardado
from ultralytics import YOLO
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm # Para mostrar una barra de progreso

# --- Configuración ---
TARGET_SIZE = 224

# --- Cargar el modelo YOLO DE SEGMENTACIÓN ---
try:
    model = YOLO('yolov8m-seg.pt') 
    print("Modelo YOLO 'yolov8m-seg.pt' cargado exitosamente. ✅")
except Exception as e:
    print(f"No se pudo cargar el modelo YOLO. Descárgalo si es necesario. Error: {e}")
    model = None

# --- MÉTODOS DE PROCESAMIENTO (CORREGIDOS) ---

def process_frame_with_margin(frame, confidence=0.4, margin=15): # CAMBIO: 'confidence_threshold' a 'confidence'
    results = model(frame, verbose=False)
    if not results or results[0].masks is None or len(results[0].masks) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    person_indices = np.where(results[0].boxes.cls == 0)[0]
    if len(person_indices) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    boxes = results[0].boxes.xywh[person_indices]
    areas = boxes[:, 2] * boxes[:, 3]
    best_detection_idx = person_indices[areas.argmax()]
    # CAMBIO: 'confidence_threshold' a 'confidence'
    if results[0].boxes.conf[best_detection_idx] < confidence:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    mask = results[0].masks[best_detection_idx].data[0].cpu().numpy().astype(np.uint8)
    mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_person = cv2.bitwise_and(frame, frame, mask=mask_resized)
    x1, y1, x2, y2 = results[0].boxes.xyxy[best_detection_idx].cpu().numpy().astype(int)
    x1, y1 = max(0, x1 - margin), max(0, y1 - margin)
    x2, y2 = min(frame.shape[1], x2 + margin), min(frame.shape[0], y2 + margin)
    cropped_segmented = segmented_person[y1:y2, x1:x2]
    if cropped_segmented.size == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    return cv2.resize(cropped_segmented, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR)

def process_frame_with_dilation(frame, confidence=0.4, kernel_size=7): # CAMBIO: 'confidence_threshold' a 'confidence'
    results = model(frame, verbose=False)
    if not results or results[0].masks is None or len(results[0].masks) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    person_indices = np.where(results[0].boxes.cls == 0)[0]
    if len(person_indices) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    boxes = results[0].boxes.xywh[person_indices]
    areas = boxes[:, 2] * boxes[:, 3]
    best_detection_idx = person_indices[areas.argmax()]
    # CAMBIO: 'confidence_threshold' a 'confidence'
    if results[0].boxes.conf[best_detection_idx] < confidence:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    mask = results[0].masks[best_detection_idx].data[0].cpu().numpy().astype(np.uint8)
    mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated_mask = cv2.dilate(mask_resized, kernel, iterations=1)
    segmented_person = cv2.bitwise_and(frame, frame, mask=dilated_mask)
    contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    main_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(main_contour)
    cropped_segmented = segmented_person[y:y+h, x:x+w]
    if cropped_segmented.size == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    return cv2.resize(cropped_segmented, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR)

def process_frame_with_hybrid(frame, confidence=0.4, kernel_size=7, margin=5): # CAMBIO: 'confidence_threshold' a 'confidence'
    results = model(frame, verbose=False)
    if not results or results[0].masks is None or len(results[0].masks) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    person_indices = np.where(results[0].boxes.cls == 0)[0]
    if len(person_indices) == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    boxes = results[0].boxes.xywh[person_indices]
    areas = boxes[:, 2] * boxes[:, 3]
    best_detection_idx = person_indices[areas.argmax()]
    # CAMBIO: 'confidence_threshold' a 'confidence'
    if results[0].boxes.conf[best_detection_idx] < confidence:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    mask = results[0].masks[best_detection_idx].data[0].cpu().numpy().astype(np.uint8)
    mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated_mask = cv2.dilate(mask_resized, kernel, iterations=1)
    segmented_person = cv2.bitwise_and(frame, frame, mask=dilated_mask)
    contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    main_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(main_contour)
    x1, y1 = max(0, x - margin), max(0, y - margin)
    x2, y2 = min(frame.shape[1], x + w + margin), min(frame.shape[0], y + h + margin)
    cropped_segmented = segmented_person[y1:y2, x1:x2]
    if cropped_segmented.size == 0:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))
    return cv2.resize(cropped_segmented, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR)
    
# --- FUNCIÓN "ROUTER" (Sin cambios) ---
def process_frame_by_method(frame, params):
    method = params.get('method', 'margin')
    if method == 'margin':
        return process_frame_with_margin(frame, **{k: v for k, v in params.items() if k != 'method'})
    elif method == 'dilation':
        return process_frame_with_dilation(frame, **{k: v for k, v in params.items() if k != 'method'})
    elif method == 'hybrid':
        return process_frame_with_hybrid(frame, **{k: v for k, v in params.items() if k != 'method'})
    else:
        return cv2.resize(frame, (TARGET_SIZE, TARGET_SIZE))

# --- Función de carga de datos (Sin cambios) ---
def load_data_from_s3_h5(s3_path):
    print(f"Cargando datos originales desde {s3_path}...")
    original_sequences, gloss_labels, video_ids_list = [], [], []
    s3 = s3fs.S3FileSystem()
    with s3.open(s3_path, 'rb') as h5_file_obj:
        with h5py.File(h5_file_obj, 'r') as hf:
            video_ids = list(hf.keys())
            for video_id in video_ids:
                group = hf[video_id]
                if 'frames' not in group or 'gloss' not in group.attrs: continue
                frames_data = group['frames'][:]
                actual_gloss_label = group.attrs.get('gloss', 'N/A')
                if actual_gloss_label == 'N/A': continue
                original_frames = [cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_GRAY2BGR) for frame in frames_data]
                original_sequences.append(np.array(original_frames))
                gloss_labels.append(actual_gloss_label)
                video_ids_list.append(video_id)
    return original_sequences, gloss_labels, video_ids_list

# --- ✨ CORREGIDO: Función para guardar los datos procesados en S3 ---
def save_data_to_s3_h5(s3_path, processed_sequences, original_labels, video_ids):
    print(f"\nGuardando {len(processed_sequences)} videos procesados en {s3_path}...")
    s3 = s3fs.S3FileSystem()
    try:
        # Usar un búfer en memoria para construir el archivo H5
        with io.BytesIO() as buffer:
            # Escribir en el búfer como si fuera un archivo
            with h5py.File(buffer, 'w') as hf:
                for i, video_id in enumerate(tqdm(video_ids, desc="Preparando guardado")):
                    group = hf.create_group(video_id)
                    frames_to_save = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in processed_sequences[i]]
                    frames_to_save_np = np.array(frames_to_save)
                    group.create_dataset('frames', data=frames_to_save_np, dtype='uint8', compression="gzip")
                    group.attrs['gloss'] = original_labels[i]
            
            # Una vez que el archivo está completo en el búfer, escribirlo de una vez en S3
            print("Subiendo archivo a S3...")
            with s3.open(s3_path, 'wb') as f:
                f.write(buffer.getvalue())

        print(f"Archivo guardado exitosamente en S3. ✅")
    except Exception as e:
        print(f"Error al guardar el archivo en S3: {e}")

# --- 🚀 Flujo de Ejecución Principal MODIFICADO ---

if __name__ == "__main__":
    if model is None:
        print("El modelo YOLO no está cargado. Terminando el script.")
    else:
        # --- 💡 CONTROL PARA PRUEBAS RÁPIDAS ---
        # Cambia a False para procesar y guardar el dataset completo.
        MODO_PRUEBA = False
        NUM_VIDEOS_PRUEBA = 5

        # 1. Definir la configuración de los datasets a procesar
        S3_INPUT_BASE = 's3://representatiohorarum/Datasets/Luego del trim/'
        S3_OUTPUT_BASE = 's3://representatiohorarum/Datasets/Luego del fondo/'
        
        datasets_to_process = [
            # Descomenta los que quieras procesar
             #{
              #   'input_name': 'ISL_trim.h5',
               #  'output_name': 'ISL_fondo.h5',
                # 'params': {'method': 'hybrid', 'kernel_size': 9, 'margin': 10, 'confidence': 0.4}
             #}
            # {
            #     'input_name': 'WLSL_trim.h5',
            #     'output_name': 'WLSL_fondo.h5',
            #     'params': {'method': 'hybrid', 'kernel_size': 9, 'margin': 10, 'confidence': 0.4}
            # },
            {
                'input_name': 'SLOVO_trim.h5',
                'output_name': 'SLOVO_fondo.h5',
                'params': {'method': 'dilation', 'kernel_size': 15, 'confidence': 0.4}
            }
        ]
        
        # 2. Bucle principal para procesar cada dataset
        for config in datasets_to_process:
            input_path = S3_INPUT_BASE + config['input_name']
            output_path = S3_OUTPUT_BASE + config['output_name']
            processing_params = config['params']
            
            print(f"\n{'='*60}")
            print(f"Iniciando procesamiento para: {config['input_name']}")
            print(f"Usando método: {processing_params['method']} con params: {processing_params}")
            if MODO_PRUEBA:
                print(f"⚠️  MODO DE PRUEBA ACTIVO: Se procesarán solo {NUM_VIDEOS_PRUEBA} videos.")
            print(f"{'='*60}")

            try:
                # Cargar datos
                original_sequences, gloss_labels, video_ids = load_data_from_s3_h5(input_path)
                
                # --- ✨ APLICAR MODO DE PRUEBA SI ESTÁ ACTIVO ---
                if MODO_PRUEBA:
                    original_sequences = original_sequences[:NUM_VIDEOS_PRUEBA]
                    gloss_labels = gloss_labels[:NUM_VIDEOS_PRUEBA]
                    video_ids = video_ids[:NUM_VIDEOS_PRUEBA]

                # --- Procesamiento del dataset para guardado ---
                print(f"\nProcesando {len(original_sequences)} videos del dataset '{config['input_name']}'...")
                all_processed_sequences = []
                for seq in tqdm(original_sequences, desc=f"Procesando {config['input_name']}"):
                    processed_sequence = [process_frame_by_method(frame, processing_params) for frame in seq]
                    all_processed_sequences.append(np.array(processed_sequence))

                # --- Guardar el resultado en un nuevo archivo H5 en S3 ---
                save_data_to_s3_h5(output_path, all_processed_sequences, gloss_labels, video_ids)

            except FileNotFoundError as e:
                print(f"Error crítico: No se encontró el archivo {input_path}. Error: {e}")
            except Exception as e:
                print(f"Ocurrió un error inesperado durante el procesamiento de {config['input_name']}: {e}")
                
        print("\nTodos los datasets han sido procesados. 🎉")

In [ ]:
import h5py
import s3fs
import matplotlib.pyplot as plt
import numpy as np

# --- Configuración ---
# Asegúrate de que esta ruta apunte al archivo que quieres verificar
s3_file_path = 's3://representatiohorarum/Datasets/Luego del fondo/ISL_fondo.h5'

print(f"🔎 Inspeccionando el archivo: {s3_file_path}\n")

try:
    s3 = s3fs.S3FileSystem()
    with s3.open(s3_file_path, 'rb') as f:
        with h5py.File(f, 'r') as hf:
            
            video_ids = list(hf.keys())
            num_videos = len(video_ids)
            
            # 1. Imprimir resumen del contenido
            print("--- RESUMEN DEL ARCHIVO H5 ---")
            print(f"Total de videos encontrados: {num_videos}")
            print("-" * 30)

            if num_videos == 0:
                print("El archivo está vacío.")
            else:
                for video_id in video_ids:
                    group = hf[video_id]
                    frames = group['frames']
                    gloss = group.attrs.get('gloss', 'N/A')
                    
                    # Imprimir información detallada de cada video
                    print(f"▶ Video ID: {video_id}")
                    print(f"  - Etiqueta (Gloss): '{gloss}'")
                    print(f"  - Número de frames: {frames.shape[0]}")
                    print(f"  - Dimensiones de frames: {frames.shape[1:]} (Alto, Ancho)")
                    print(f"  - Tipo de datos: {frames.dtype}")
            
            # 2. Visualizar todos los frames de todos los videos
            if num_videos > 0:
                print("\n--- VISUALIZACIÓN DE TODOS LOS FRAMES ---")
                
                for video_id in video_ids:
                    frames_data = hf[video_id]['frames'][:]
                    gloss = hf[video_id].attrs.get('gloss', 'N/A')
                    num_frames = len(frames_data)
                    
                    # Crear una figura con subplots para todos los frames del video
                    # Usar una cuadrícula con un máximo de 5 columnas para mejor legibilidad
                    max_cols = 5
                    num_rows = (num_frames + max_cols - 1) // max_cols  # Calcular filas necesarias
                    fig, axes = plt.subplots(num_rows, min(num_frames, max_cols), 
                                             figsize=(4 * min(num_frames, max_cols), 4 * num_rows))
                    
                    # Asegurarse de que axes sea una matriz 2D para facilitar la indexación
                    if num_frames == 1:
                        axes = np.array([[axes]])
                    elif num_rows == 1:
                        axes = np.array([axes])
                    
                    # Iterar sobre todos los frames
                    for j in range(num_frames):
                        row = j // max_cols
                        col = j % max_cols
                        ax = axes[row, col]
                        ax.imshow(frames_data[j], cmap='gray')
                        ax.axis('off')
                        ax.set_title(f"{video_id} | '{gloss}'\nFrame: {j}")
                    
                    # Ocultar ejes vacíos si el número de frames no llena la cuadrícula
                    if num_frames < num_rows * max_cols:
                        for j in range(num_frames, num_rows * max_cols):
                            row = j // max_cols
                            col = j % max_cols
                            axes[row, col].axis('off')
                    
                    plt.suptitle(f"Video: {video_id} | Gloss: {gloss}")
                    plt.tight_layout()
                    plt.show()

except FileNotFoundError:
    print(f"❌ ERROR: No se encontró el archivo en la ruta: {s3_file_path}")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")

In [2]:
import cv2
import numpy as np
import h5py
import s3fs
import os
import time
from tqdm import tqdm # Añadido para una mejor visualización del progreso

def repair_black_frames():
    """
    Busca y reemplaza frames completamente negros en archivos H5 de videos.
    Un frame negro se reemplaza por una fusión del frame no-negro anterior y el siguiente.
    """

    # --- 💡 CONFIGURACIÓN DE RUTAS (SECCIÓN MODIFICADA) ---
    INPUT_BASE_PATH = "s3://representatiohorarum/Datasets/Luego del fondo/"
    OUTPUT_BASE_PATH = "s3://representatiohorarum/Datasets/luego de frames negros/"

    # Define los datasets a procesar de forma organizada
    datasets_to_process = {
        "ISL":   {"input": "ISL_fondo.h5", "output": "ISL_negro.h5"},
        "SLOVO": {"input": "SLOVO_fondo.h5", "output": "SLOVO_negro.h5"},
        #"WLSL":  {"input": "WLSL_fondo.h5", "output": "WLSL_negro.h5"},
    }

    # Construye las listas de archivos de entrada y salida dinámicamente
    h5_input_files = [INPUT_BASE_PATH + info["input"] for info in datasets_to_process.values()]
    h5_output_files = [OUTPUT_BASE_PATH + info["output"] for info in datasets_to_process.values()]
    # --- FIN DE LA SECCIÓN MODIFICADA ---
    

    print("--- Iniciando el proceso de verificación y reparación de frames negros ---")

    try:
        s3 = s3fs.S3FileSystem()
    except Exception as e:
        print(f"❌ ERROR: No se pudo inicializar S3FileSystem. Verifica tus credenciales.")
        print(f"Detalle del error: {e}")
        return

    # Iterar sobre cada par de archivos
    for input_path, output_path in zip(h5_input_files, h5_output_files):
        print(f"\n{'='*60}")
        print(f"▶️  Procesando archivo: {os.path.basename(input_path)}")
        print(f"💾 Guardando en: {os.path.basename(output_path)}")
        print(f"{'='*60}")
        
        temp_filename = f"temp_repair_{os.path.basename(output_path)}_{int(time.time())}.h5"
        total_repaired_frames = 0
        total_repaired_videos = 0

        try:
            with s3.open(input_path, 'rb') as s3_file:
                with h5py.File(s3_file, 'r') as hf_in:
                    with h5py.File(temp_filename, 'w') as hf_out:
                        video_ids = list(hf_in.keys())
                        print(f"Se encontraron {len(video_ids)} videos. Verificando...")

                        for video_id in tqdm(video_ids, desc="Procesando videos"):
                            group_in = hf_in[video_id]

                            if 'frames' not in group_in:
                                continue

                            frames = group_in['frames'][:]
                            
                            group_out = hf_out.create_group(video_id)
                            for key, value in group_in.attrs.items():
                                group_out.attrs[key] = value

                            # --- Lógica de búsqueda y reemplazo (sin cambios) ---
                            black_frame_indices = [idx for idx, frame in enumerate(frames) if not np.any(frame)]
                            
                            if black_frame_indices:
                                total_repaired_videos += 1
                                total_repaired_frames += len(black_frame_indices)
                                
                                frames_list = [frame.astype(np.uint8) for frame in frames]

                                for idx in black_frame_indices:
                                    prev_frame_idx = next((prev_i for prev_i in range(idx - 1, -1, -1) if prev_i not in black_frame_indices), -1)
                                    next_frame_idx = next((next_i for next_i in range(idx + 1, len(frames_list)) if next_i not in black_frame_indices), -1)

                                    if prev_frame_idx != -1 and next_frame_idx != -1:
                                        prev_frame = frames_list[prev_frame_idx]
                                        next_frame = frames_list[next_frame_idx]
                                        new_frame = cv2.addWeighted(prev_frame, 0.5, next_frame, 0.5, 0)
                                        frames_list[idx] = new_frame
                                    elif prev_frame_idx != -1:
                                        frames_list[idx] = frames_list[prev_frame_idx]
                                    elif next_frame_idx != -1:
                                        frames_list[idx] = frames_list[next_frame_idx]
                                
                                frames = np.array(frames_list, dtype=np.uint8)

                            # Guardar el dataset de frames (modificado o no)
                            group_out.create_dataset('frames', data=frames, compression="gzip")
            
            print(f"\n✅ Verificación completada.")
            print(f"  - Se repararon {total_repaired_frames} frames en {total_repaired_videos} videos.")
            
            print(f"⏫ Subiendo archivo procesado a: {output_path}")
            s3.put(temp_filename, output_path)
            print("   Carga completa.")

        except Exception as e:
            print(f"❌ ERROR: Ocurrió un error inesperado al procesar '{input_path}': {e}")
        finally:
            if os.path.exists(temp_filename):
                os.remove(temp_filename)
                #print(f"   Archivo temporal '{temp_filename}' eliminado.")

    print("\n\n--- Proceso de reparación de frames completado para todos los archivos. --- 🎉")

# --- Ejecutar la función ---
if __name__ == '__main__':
    repair_black_frames()

--- Iniciando el proceso de verificación y reparación de frames negros ---

▶️  Procesando archivo: ISL_fondo.h5
💾 Guardando en: ISL_negro.h5
Se encontraron 1355 videos. Verificando...


Procesando videos: 100%|██████████| 1355/1355 [00:23<00:00, 57.25it/s]



✅ Verificación completada.
  - Se repararon 0 frames en 0 videos.
⏫ Subiendo archivo procesado a: s3://representatiohorarum/Datasets/luego de frames negros/ISL_negro.h5
   Carga completa.

▶️  Procesando archivo: SLOVO_fondo.h5
💾 Guardando en: SLOVO_negro.h5
Se encontraron 2120 videos. Verificando...


Procesando videos: 100%|██████████| 2120/2120 [00:37<00:00, 56.53it/s]



✅ Verificación completada.
  - Se repararon 0 frames en 0 videos.
⏫ Subiendo archivo procesado a: s3://representatiohorarum/Datasets/luego de frames negros/SLOVO_negro.h5
   Carga completa.


--- Proceso de reparación de frames completado para todos los archivos. --- 🎉


In [ ]:
import h5py
import s3fs
import matplotlib.pyplot as plt
import numpy as np

# --- Configuración ---
S3_BASE_PATH = "s3://representatiohorarum/Datasets/luego de frames negros/"
FILENAMES = [#"ISL_negro.h5", 
             "SLOVO_negro.h5" 
             #"WLSL_negro.h5"
                ]

# Puedes ajustar cuántos frames por video quieres ver
NUM_FRAMES_PER_VIDEO = 4  # Muestra el primer, medio y último frame

print("--- Iniciando la verificación visual de los datasets procesados ---")

# Inicializa el sistema de archivos de S3
s3 = s3fs.S3FileSystem()

# Itera sobre cada archivo final
for filename in FILENAMES:
    s3_path = S3_BASE_PATH + filename
    print(f"\n{'='*70}")
    print(f"🔎 Analizando archivo: {filename}")
    print(f"{'='*70}")

    try:
        with s3.open(s3_path, 'rb') as s3_file:
            with h5py.File(s3_file, 'r') as hf:
                all_video_ids = list(hf.keys())

                if not all_video_ids:
                    print("‼️  Este archivo no contiene videos.")
                    continue
                
                print(f"Mostrando todos los {len(all_video_ids)} videos...")

                # Prepara la figura para mostrar los frames de todos los videos
                fig, axes = plt.subplots(
                    len(all_video_ids), 
                    NUM_FRAMES_PER_VIDEO, 
                    figsize=(10, 3.5 * len(all_video_ids))
                )
                
                # Asegura que 'axes' sea siempre 2D para una fácil indexación
                axes = np.atleast_2d(axes)

                # Itera sobre todos los videos para mostrarlos
                for i, video_id in enumerate(all_video_ids):
                    frames = hf[video_id]['frames'][:]
                    gloss = hf[video_id].attrs.get('gloss', 'N/A')
                    
                    # Selecciona los índices del primer, medio y último frame
                    indices_to_plot = sorted(list(set([0, len(frames) // 2, len(frames) - 1])))
                    
                    for j, frame_idx in enumerate(indices_to_plot):
                        ax = axes[i, j]
                        ax.imshow(frames[frame_idx], cmap='gray')
                        ax.set_title(f"{video_id}\n'{gloss}' | Frame {frame_idx}")
                        ax.axis('off')
                    
                    # Oculta los ejes no utilizados si un video tiene menos de 3 frames
                    for j in range(len(indices_to_plot), NUM_FRAMES_PER_VIDEO):
                        axes[i, j].axis('off')
                        
                fig.suptitle(f"Muestra de Frames para '{filename}'", fontsize=16, y=1.02)
                plt.tight_layout()
                plt.show()

    except FileNotFoundError:
        print(f"❌ ERROR: No se encontró el archivo en la ruta: {s3_path}")
    except Exception as e:
        print(f"❌ Ocurrió un error inesperado al procesar {filename}: {e}")

Matplotlib is building the font cache; this may take a moment.


--- Iniciando la verificación visual de los datasets procesados ---

🔎 Analizando archivo: SLOVO_negro.h5
Mostrando todos los 2120 videos...
